In [4]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random

In [ ]:
# TODO: Get dataset from external url

In [8]:
df_train = pd.read_csv('dataset/train.csv',
                         sep=',',
                         names = ['user_id','item_id','rating'],
                         header=None)

df_items = pd.read_csv('dataset/anime.csv',
                        sep=',',
                        names = ['anime_id','name','genre','type','episodes','rating','members'],
                        header=None,
                        encoding='latin-1')

df_train.head()

,user_id,item_id,rating
0,20881,1536,9
1,25996,1241,8
2,25409,8668,7
3,29728,6325,5
4,71886,1887,9


In [5]:
unique_user_ids = df_train['user_id'].unique()
unique_item_ids = df_train['item_id'].unique()

user_id_map = {uid: idx for idx, uid in enumerate(unique_user_ids)}
item_id_map = {iid: idx for idx, iid in enumerate(unique_item_ids)}
reverse_item_id_map = {idx: iid for iid, idx in item_id_map.items()} 

rows = []
cols = []
data = []

for row in df_train.itertuples(index=False):
    user_idx = user_id_map[row.user_id]
    item_idx = item_id_map[row.item_id]

    rows.append(user_idx)
    cols.append(item_idx)
    data.append(1)

user_item_matrix = sparse.csr_matrix((data, (rows, cols)), shape=(len(user_id_map), len(item_id_map)))

In [6]:
def recommend_random(user_idx, user_items, N=10):
    seen_items = set(user_items.indices)
    all_items = set(range(user_item_matrix.shape[1]))
    unseen_items = list(all_items - seen_items)

    if len(unseen_items) < N:
        N = len(unseen_items)

    recommended_items = random.sample(unseen_items, N)
    return recommended_items

def show_recommendations_random(user_id, n):
    user_idx = user_id_map[user_id]
    user_items = user_item_matrix[user_idx]
    rec = recommend_random(user_idx, user_items, N=n)
    return df_items[df_items['anime_id'].isin([reverse_item_id_map[i] for i in rec])]['name']

In [9]:
print(show_recommendations_random(user_id=20881, n=10))


26                    Tengen Toppa Gurren Lagann
128                                    Toradora!
369                        Initial D Fifth Stage
527                        Little Witch Academia
1575    Sakasama no Patema: Beginning of the Day
2440                           Fushigi Yuugi OVA
2926             Shinmai Maou no Testament Burst
3010                           Asura Cryin&#039;
3829                      Ladies versus Butlers!
6963                        PES: Peace Eco Smile
Name: name, dtype: object
